In [1]:
# Not really used anymore
class Document:
    '''A document object is a dictionary'''
    # Constructer method
    def __init__(self, **dictionary_entries):
        '''Initiates Document object with values from dictionary'''
        # [id, article_url, title, author, publised_date, contents, type, source]
        self.__dict__.update(dictionary_entries)
    
    # Getters (Should update: https://docs.python.org/2/library/functions.html#property)
    def id(self):
        return self.id
    def article_url(self):
        return self.article_url
    def title(self):
        return self.title
    def author(self):
        return self.author
    def published_date(self):
        return self.published_date
    def contents(self):
        return self.contents
    def type(self):
        return self.type
    def source(self):
        return self.source
    
    # Method for printing the object
    def __str__(self):
        # TODO: implement printing function
        return "Document Identifier: " + self.id()
        

In [23]:
# RUN: 'pip install jsonlines' on: ModuleNotFoundError: No module named 'jsonlines'
import jsonlines
from tqdm import tqdm
import pandas as pd
import pickle

# RUN: 'pip install whoosh' on: ModuleNotFoundError: No module named 'whoosh'
import os, os.path

#from whoosh import index
import whoosh
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.writing import AsyncWriter

from datetime import datetime
from pytz import timezone

######################################

class Indexer:
    '''This class is used to create index of the WP database'''
    
    index_created = False
    index_directory = ""
    data_path = ""       
    
    def __init__(self):
        print('Indexer has been initiated.')
        
    def indexCreated(self):
        return self.index_created
    
    def openIndex(self):
        pass
      
    ## Count documents
    def countDocuments(self):
        print("Counting documents, this might take a while...")
        counter = 0
        tenKCounter = 0

        with jsonlines.open(self.data_path) as reader:
            for obj in tqdm(reader.iter(type=dict, skip_invalid=True)):
                tenKCounter += 1
                counter += 1
            
                if tenKCounter >= 10000:
                    print("Current count is: " + str(counter))
                    tenKCounter = 0 
        print("Last count is: " + str(counter))
        print("Counting done.")
    
    def printDocumentData(self):
        '''Prints every 10000th object'''
        #dataframe = pd.DataFrame(columns = ['Title', 'Content', 'Year'])
        with jsonlines.open(self.data_path) as reader:
            counter = 0
            for obj in tqdm(reader.iter(type=dict, skip_invalid=True)):
                counter += 1
                if (counter % 10000) == 0:
                    print("Object number:" + str(counter))
                    print(type(obj))
                    doc = Document(**obj)
                    print(doc)
        
    def extractDocumentContents(self, contents):
        '''Extracts document contents from array of dicts to string'''
        #TODO IMPLEMENT
        return ""
    
    def extractDocumentDate(self, epochTimestamp):
        '''Converst UNIX epoch timestamp to DATETIME'''
        # TODO IMPLENT Default value on Handling Error: ValueError: invalid literal for int() with base 10: 'None'
        if (epochTimestamp == ''):
            return datetime(1,1,1)
        try:
            # Test if source is within specified dates
            if 1512000000 < epochTimestamp < 1543536000:
                date_info = epochTimestamp
                removed_zeros = str(date_info)[0:10]
                timestamp = int(removed_zeros)
                return datetime.fromtimestamp(timestamp, timezone('EST'))
            else:
                return datetime(1,1,1)
        except:
            print("ERROR: In extractDocumentDate()")
            print("ERROR: ", sys.exc_info()[0])
            print("Caused by value: " + str(date_info))
            return datetime(1,1,1)
    
    def setIndexDirectory(self, directory):
        self.index_directory = directory
    
    def getIndexLocation(self):
        return self.index_directory
    
    def setDataPath(self, file_location):
        self.data_path = file_location
    
    def getDataPath(self):
        return self.data_path
    
    def setIndexCreatedTrue(self):
        self.index_created = True
        
    
    def index(self):
        '''This method indexes the data'''
        # https://whoosh.readthedocs.io/en/latest/api/writing.html
        # https://appliedmachinelearning.blog/2018/07/31/developing-a-fast-indexing-and-full-text-search-engine-with-whoosh-a-pure-python-library/
        # TODO add document contents to index
        
        #  [id, article_url, title, author, publised_date, contents, type, source]
        # Define fields using whoosh's 'Schema' | https://whoosh.readthedocs.io/en/latest/schema.html#
        # Can add field boost here
        # TODO: critically look at different fields, which are important?, which need to be shown to the user? etc.
        # TODO: check if index_location has been set
        
        assert self.index_directory and self.data_path # Both variables have to be set
        
        schema = Schema(doc_id = whoosh.fields.ID(unique=True, stored=True),\
                       article_url = whoosh.fields.STORED,\
                       title = whoosh.fields.TEXT(stored=True),\
                       author = whoosh.fields.ID,\
                       published_date = whoosh.fields.DATETIME,\
                       contents = whoosh.fields.TEXT,\
                       type = whoosh.fields.STORED,\
                       source = whoosh.fields.STORED)
              
        # Creating a index writer to add document as per schema
        myindex = whoosh.index.create_in(self.index_directory,schema)
        writer = whoosh.writing.AsyncWriter(myindex)
        
        # Loop over data
        print("Looping over data. Indexing each article.")
        print("This might take a few minutes...")
        counter = 0
        checker = 0
        with jsonlines.open(self.data_path) as reader:
            for obj in tqdm(reader.iter(type=dict, skip_invalid=True)):
                retreived_date = self.extractDocumentDate(obj['published_date'])
                if retreived_date != datetime(1,1,1):
                    counter = counter + 1
                    writer.add_document(doc_id=obj['id'],\
                                        article_url=obj['article_url'],\
                                        title=obj['title'],\
                                        author=obj['author'],\
                                        published_date=retreived_date,\
                                        contents = self.extractDocumentContents(obj['contents']),\
                                        type=obj['type'],\
                                        source=obj['source'])
                if (counter > checker) and at_least_one:
                    writer.commit()
                    checker = checker + 100
                    print("Indexed " + str(counter) + " articles")
            
            writer.commit()
            print("Indexed " + str(counter) + " articles")
            print("Looping complete.")
        print("Index created!")
        self.setIndexCreatedTrue()
        
    


# indexer = Indexer()
# #indexer.countDocuments()
# #indexer.printDocumentData()
# #indexer.createDocumentDataStore()
# indexer.index()

In [29]:
###############################################
from whoosh.qparser import QueryParser
from whoosh import scoring
from whoosh.index import open_dir

class Ranking:
    '''This class contains functions that are used to create a ranking based on different algorithms'''
    
    show_n_results = 3
    index_directory = ""
    
    def __init__(self):
        print("Ranking class has been initiated.")
    
    def indexCreated(self):
        return indexCreated
    
    def createIndex(self):
        pass
    
    def setIndexDirectory(self, directory):
        self.index_directory = directory
    
    def openIndex(self):
        assert self.index_directory
        return open_dir(self.index_directory)
    
    def printResults(self, results):
        # https://whoosh.readthedocs.io/en/latest/api/searching.html
        for result in results:
            print("Result type: " + str(type(result)))
            print("Result: " + str(result))
            print("Result fields: " + str(result.fields()))
            print()
            
    
    def searchTermFrequency(self, user_query):
        #TODO check if index has been created
        index = self.openIndex()
        
        with index.searcher(weighting=scoring.Frequency) as searcher:
            parsed_query = QueryParser("title", index.schema).parse(user_query)
            results = searcher.search(parsed_query, limit=self.show_n_results)
            self.printResults(results)
    
    def searchTF_IDF():
        pass
    
    def searchBM25F():
        pass
        

# ranking = Ranking()
# ranking.searchTermFrequency("Trump election")
    

In [30]:
################################################
class Evaluation:
    '''Used to evaluate the ranking results'''
    #should check if query used is in the 

In [31]:
##################################################
class UserInterface:
    '''Class used to handle user interaction'''
    # TODOS:
    # implement method that asks used if 
    # implement methods that asks the user to set different options
    # 
          
    def __init__(self):
        '''This is the constructor method of the UI'''
        print("UserInterface has been initiated.")       
        
    def getUserQuery(self):
        ''''Retrieves the query from the user and returns it'''
        userInput = input("Please enter a query: ")
        return userInput
    
    def indexAlreadyCreated(self, directory_location):
        print("Index is set.")
        print("Index files are stored in directory:" + directory_location)
        return
    
    def shouldCreateIndex(self):
        print("Do you wish to create a new index?")
        userInput = input("Answer [y/n]: ")
        if userInput == ('y' or 'Y' or "yes"):
            return True
        return False
    
    def indexIsNotSet(self):
        print("Index has not been set.")
        return
    
    def getIndexDirectory(self, default_directory):
        userInput = input("Please enter a directory name for the index (default = /"+ default_directory + "): ")
        if userInput == "":
            return default_directory
        return userInput
    
    def getDataPath(self, default_data_path):
        print("Please enter the path to the TREC_Washington_Post_collection.v2.jl file.")
        print("The default location is: /"+ default_data_path + "): ")
        userInput = input()
        if userInput == "":
            return default_data_path
        return userInput
    
    def creatingIndex(self, index_directory, data_path):
        pass
    
    def shouldAddExistingIndex(self):
        print("Do you wish to add an existing index?")
        userInput = input("Answer [y/n]: ")
        if userInput == ('y' or 'Y' or "yes"):
            return True
        return False
    
    def stopSearchEngine(self):
        print("Search Engine is stopped.")
        
    def shouldTerminateSearchEngine(self):
        print("Do you wish to stop the search engine?")
        userInput = input("Answer [y/n]: ")
        if userInput == ('y' or 'Y' or "yes"):
            return True
        return False
    

In [32]:
# Block used to run everything:
# TODO expand this 'Controll' class that handles information flow
# TODO add querying functionality
# TODO implement choice of search algorithm
# TODO implement evaluation mode
import time

class SearchEngine:
    '''This class embodies the search engine and acts a a controller class'''
    
    UI = None
    Indexer = None
    Ranking = None
    
    RUNNING = False
    STOPPED = False
    EVALUATION_MODE = False
    USER_MODE = True
    
    user_query = ""
    DEFAULT_data_path = 'WP-corpus/data/TREC_Washington_Post_collection.v2.jl'
    DEFAULT_index_directory = "indexdir"
    
    
    def __init__(self):
        print("Search Engine has been initiated.")
    
    def stopSearchEngine(self, UI):
        self.RUNNING = False
        UI.stopSearchEngine()        
        
    def setIndex(self, Indexer, UI):
        if Indexer.indexCreated():
            directory_location = Indexer.getIndexLocation()
            UI.indexAlreadyCreated(directory_location)
            return
        else:
            UI.indexIsNotSet()
            if UI.shouldCreateIndex(): # This is for creating a new index
                # Index directory (setDir)
                index_directory = UI.getIndexDirectory(self.DEFAULT_index_directory)
                Indexer.setIndexDirectory(index_directory)
                # Check if index_dir exist and makes one if it doesn't
                if not os.path.exists(index_directory):
                    os.mkdir(index_directory)
                
                # Data file path (setPath)    
                data_path = UI.getDataPath(self.DEFAULT_data_path)
                Indexer.setDataPath(data_path)
                
                # Creating Index
                UI.creatingIndex(index_directory, data_path)
                Indexer.index()
                return
                
            elif UI.shouldAddExistingIndex(): # This is for adding an existing index
                
                # TODO communicate chosen directories and chosen data file
                # TODO add checks to check for existence of index?
                
                # Index directory, duplicate code -> (setDir)
                index_directory = UI.getIndexDirectory(self.DEFAULT_index_directory)
                Indexer.setIndexDirectory(index_directory)
                # Check if index_dir exist and makes one if it doesn't
                if not os.path.exists(index_directory):
                    os.mkdir(index_directory)
                
                # Data file path, duplicate code -> (setPath)    
                data_path = UI.getDataPath(self.DEFAULT_data_path)
                Indexer.setDataPath(data_path)
                
                # Set index created to true
                Indexer.setIndexCreatedTrue()
                return
                                
            else: # TODO: Ask if the user wants to continue or stop
                if UI.shouldTerminateSearchEngine(): 
                    self.STOPPED = True
                return
            return
       
    def run(self):
        '''This function start the search engine'''
        print('Search Engine started.')
        
        self.UI = UserInterface()
        self.Indexer = Indexer()
        self.Ranking = Ranking()
                
        self.RUNNING = True
        while self.RUNNING:
            if not self.Indexer.indexCreated():
                #TODO Inform user that an index needs to be set
                self.setIndex(self.Indexer, self.UI)
                if self.STOPPED == True: # Stop SE if user did not want to continue
                    self.stopSearchEngine(self.UI)
                    break
            #TODO ask user to choose a mode
                       
            if self.EVALUATION_MODE:
                # evaluation = Evaluation()
                # This is used to evaluate SE against the TREC relevance judgements
                # TODO load evaluation document and queries
                # TODO pass queries on to ranking and obtain ranking results
                # TODO compare ranking results to TREC evaluation
                # TODO display results, how?
                # TODO test different algorithms
                pass
            if self.USER_MODE:
                # This is used to query questions
                #TODO ask for search algorithm
                #TODO ask for user query
                #TODO move printing results to UI
                print("Entering USER_MODE")
                user_query = self.UI.getUserQuery()
                print("Entered query: " + user_query)
                
                index_dir = self.Indexer.getIndexLocation()
                self.Ranking.setIndexDirectory(index_dir)
                self.Ranking.searchTermFrequency(user_query)
                
            print("Starting over.")
            time.sleep(1)

In [33]:
mySearchEngine = SearchEngine()
mySearchEngine.run()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 'Kernelspec' module not installed in the selected interpreter (C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe).
 Please re-install or update 'jupyter'.